In [ ]:

import os
import json
import redis
import numpy as np
import pandas as pd
from sklearn import metrics

In [2]:
# Connect to redis

secret_file_path = ""
backend = redis.Redis(
    charset="utf-8",
    decode_responses=True,
    password=open(secret_file_path).read().strip(),
    port=6666,
)
key_values = {key: json.loads(backend.get(key)) for key in backend.keys()}

OUTPUT_DIR = ""

In [5]:
# Retrieve user data from database and save as jsonl

# Filter user ids during debugging
user_ids = [k for k in key_values if ":user:" in k and "debug" not in k]
user_data = []

for user in user_ids:
    tmp = json.loads(backend[user])

    experiment_id = user.split(":")[0]
    user_id = user.split(":")[2]
    if len(tmp['split_id']) > 1: split_id = int(tmp['split_id'][0])
    else: split_id = int(tmp['split_id'])
    
    if 'consent-check' in tmp: consent_check = tmp['consent-check']
    else: consent_check = False

    if 'attention-check' in tmp: attention_check = tmp['attention-check']
    else: attention_check = False
    if 'attention-state' in tmp: attention_state = tmp['attention-state']
    else: attention_state = False

    if 'user-task-instances' in tmp: user_task_instances = tmp['user-task-instances']
    else: user_task_instances = False

    if 'task-state' in tmp: task_state = tmp['task-state']
    else: task_state = False

    if 'survey-check' in tmp: survey_check = tmp['survey-check']
    else: survey_check = False
    if 'survey-state' in tmp: survey_state = tmp['survey-state']
    else: survey_state = False

    user_data.append(
        {
            "experiment_id": experiment_id,
            "user_id": user_id,
            "split_id": split_id,
            "consent_check": consent_check,
            "attention_check": attention_check,
            "attention_state": attention_state,
            "user_task_instances": user_task_instances,
            "task_state": task_state,
            "survey_check": survey_check,
            "survey_state": survey_state,
        }
    )

user_df = pd.DataFrame(user_data)
user_data_path = ""
user_df.to_json(os.path.join(OUTPUT_DIR, user_data_path), lines=True, orient="records")

In [ ]:
# Filter old data and include only experiment data

df = pd.read_json(user_data_path, lines=True)

# This line excludes rows with the following criteria:
# 1. attention_state, task state, survey state are false (we only want complete data)
# 2. remove user ids
filtered_df = df[(~df['user_id'].str.startswith('0')) & \
                 ~(df['attention_state'] == False) & \
                 ~(df['task_state'] == False) & \
                 ~(df['survey_state'] == False)]

print(f"len of filtered_df: {len(filtered_df)}")


In [8]:
def create_senstitive_attribute_dict(key):
    tmp = json.loads(backend[key])
    assert len(tmp) == 200
    dataset = key.split(":")[0].split("-")[0]
    
    d = {}
    for i in tmp:
        sensitive_attribute = ''
        # gender for German Credit and race for rcdv
        if dataset == 'rcdv':
            sensitive_attribute = i['features']['white']
        if dataset == 'german':
            if i['features']['personal-status-sex_1'] == 1 or i['features']['personal-status-sex_3'] == 1:
                sensitive_attribute = 1 # male
            if i['features']['personal-status-sex_2'] == 1 or i['features']['personal-status-sex_5'] == 1:
                sensitive_attribute = 0 # female
        instance_id = i['instance-idx']
        d[instance_id] = sensitive_attribute
    return d


# Create task data senstive attributes dict
# to retrieve sensitive attributes more easily

task_data = [k for k in key_values if "task-data" in k and "debug" not in k]
task_data_dict = {}
for key in task_data:
    tmp_d = create_senstitive_attribute_dict(key)
    experiment_id = key.split(":")[0]
    task_data_dict[experiment_id] = tmp_d

In [ ]:
# Save each user label as a row in a dataframe

user_ids = set()

task_data = []
for idx, row in filtered_df.iterrows():
    task_labels = row['task_state']['labels']
    try:
        assert len(task_labels) == 20
    except:
        print(f"user id: {row['user_id']}")

    # Get time label on last task label
    task_keys = list(task_labels.keys())
    last_task_key = task_keys[len(task_keys)-1]
    last_task_end_time = task_labels[last_task_key]['label-time']

    # Get time label on last attention label
    attention_labels = row['attention_state']['labels']
    attention_qns_keys = list(attention_labels.keys())
    last_attention_qn_key = attention_qns_keys[len(attention_qns_keys)-1]
    attention_end_time = attention_labels[last_attention_qn_key]['label-time']

    # Find total time taken and average time taken in seconds
    # Divide by 100 since original timestamps are in milliseconds
    total_time_taken_for_task = (last_task_end_time - attention_end_time) / 100 / 60
    average_time_per_prediction = total_time_taken_for_task / len(task_labels)

    count = 0
    for index, task_label in task_labels.items():
        # Use only the first 20 labels
        if count > 20:
            break

        if 'instance' not in task_label:
            break

        instance_id = int(task_label['instance'])
        experiment_id = row['experiment_id']

        user_label = 0
        try:
            if task_label['label'] == "will_recidivate" or task_label['label'] == "good":
                user_label = 1
        except:
            user_ids.add(row['user_id'])
            break
        actual_label = int(task_label['actual_label'])
        prediction = int(task_label['prediction'])

        task_data.append(
            {
                "experiment_id": experiment_id,
                "user_id": row['user_id'],
                "split_id": row['split_id'],
                "instance_order": index,
                "instance_id": instance_id,
                "user_label": user_label,
                "actual_label": actual_label,
                "prediction": prediction,
                "sensitive_attribute": task_data_dict[experiment_id][instance_id],
                "average_time": average_time_per_prediction
            }
        )
        count += 1

task_df = pd.DataFrame(task_data)
task_data_path = ""
task_df.to_json(os.path.join(OUTPUT_DIR, task_data_path), lines=True, orient="records")



In [12]:
# Analyze task data with evaluation metrics

def calculate_tpr_fpr(y_true, y_pred):
    cm = metrics.confusion_matrix(y_true, y_pred)
    if cm.shape == (1, 1):
        # Only one class present in y_true
        tn, fp, fn, tp = 0, 0, 0, cm[0, 0]
    elif cm.shape == (2, 2):
        tn, fp, fn, tp = cm.ravel()
    else:
        raise ValueError("Unexpected confusion matrix shape: {}".format(cm.shape))
    
    # Avoid division by zero when calculating TPR and FPR
    tpr = tp / (tp + fn) if tp + fn > 0 else 0
    fpr = fp / (fp + tn) if fp + tn > 0 else 0
    return round(tpr, 3), round(fpr, 3)


def absolute_equality_of_opportunity_difference(y_true, y_pred, sensitive_attr):
    group_0_indices = (sensitive_attr == 0)
    group_1_indices = (sensitive_attr == 1)
    
    tpr_0, _ = calculate_tpr_fpr(y_true[group_0_indices], y_pred[group_0_indices])
    tpr_1, _ = calculate_tpr_fpr(y_true[group_1_indices], y_pred[group_1_indices])
    
    return np.abs(tpr_0 - tpr_1)


def absolute_equalized_odds_difference(y_true, y_pred, sensitive_attr):
    group_0_indices = (sensitive_attr == 0)
    group_1_indices = (sensitive_attr == 1)
    
    tpr_0, fpr_0 = calculate_tpr_fpr(y_true[group_0_indices], y_pred[group_0_indices])
    tpr_1, fpr_1 = calculate_tpr_fpr(y_true[group_1_indices], y_pred[group_1_indices])
    
    return (np.abs(tpr_0 - tpr_1) + np.abs(fpr_0 - fpr_1)) / 2


conditions = task_df.apply(lambda row: f"{row['experiment_id']}_{row['split_id']}", axis=1).unique()
objective_results = ""
with open(objective_results, "w") as file:
    file.write(f"total conditions: {len(conditions)}\n\n")

    for condition in conditions:
        experiment_id = condition.split('_')[0]
        split_id = condition.split('_')[1]
        tmp_df = task_df[(task_df['experiment_id'] == experiment_id) & (task_df['split_id'] == int(split_id))]

        time_df = tmp_df.groupby(['experiment_id', 'split_id','user_id'])['average_time'].mean().reset_index(name='average_time_per_prediction')
        time_taken = np.mean(time_df['average_time_per_prediction'])

        over_reliance = round(len(tmp_df[(tmp_df['user_label'] == tmp_df['prediction']) & (tmp_df['actual_label'] != tmp_df['prediction'])]) / len(tmp_df), 3)
        under_reliance = round(len(tmp_df[(tmp_df['user_label'] != tmp_df['prediction']) & (tmp_df['actual_label'] == tmp_df['prediction'])]) / len(tmp_df), 3)

        preds = tmp_df['prediction']
        labels = tmp_df['actual_label']
        user_labels = tmp_df['user_label']
        sensitive_attr = tmp_df['sensitive_attribute']
        assert len(preds) == len(labels) == len(user_labels) == len(tmp_df)
        
        accuracy = round(metrics.accuracy_score(labels, preds), 3)
        precision = round(metrics.precision_score(labels, preds), 3)
        recall = round(metrics.recall_score(labels, preds), 3)
        f1 = round(metrics.f1_score(labels, preds), 3)
        tn, fp, fn, tp = metrics.confusion_matrix(labels, preds).ravel()

        tpr, fpr = calculate_tpr_fpr(labels, preds)
        aeood = round(absolute_equality_of_opportunity_difference(labels, preds, sensitive_attr), 3)
        aeod = round(absolute_equalized_odds_difference(labels, preds, sensitive_attr), 3)

        file.write(f"===== {experiment_id}:{split_id} ======\n")
        file.write(f"# of participants: {len(preds) / 20} (we want it to be around 30)\n")
        file.write(f"# of preds/labels: {len(preds)} (we want it to be around 600)\n")
        file.write(f"accuracy: {accuracy}\n")
        file.write(f"precision: {precision}\n")
        file.write(f"recall: {recall}\n")
        file.write(f"f1: {f1}\n")
        file.write(f"over reliance: {over_reliance}\n")
        file.write(f"under reliance: {under_reliance}\n")
        file.write(f"tpr: {tpr}\n")
        file.write(f"fpr: {fpr}\n")
        file.write(f"absolute equality of opportunity difference: {aeood}\n")
        file.write(f"absolute equalized odds difference: {aeod}\n")
        file.write(f"true negatives: {tn}\n")
        file.write(f"false positives: {fp}\n")
        file.write(f"false negatives: {fn}\n")
        file.write(f"true positives: {tp}\n")
        file.write(f"time taken per prediction in minutes: {time_taken}\n")
        file.write("\n")

In [13]:
# Save each survey label as a row in a dataframe

likert_score_conversion = {
    "Strongly Disagree": 1,
    "Disagree": 2,
    "Neutral": 3,
    "Agree": 4,
    "Strongly Agree" : 5,
}

survey_data = []
for idx, row in filtered_df.iterrows():
    survey_labels = row['survey_state']['labels']

    for index, survey_label in survey_labels.items():
        instance_id = int(task_label['instance'])
        experiment_id = row['experiment_id']
        survey_label = survey_label['label']

        survey_data.append(
            {
                "experiment_id": experiment_id,
                "user_id": row['user_id'],
                "split_id": row['split_id'],
                "question_number": index,
                "survey_label": survey_label,
                "survey_label_score": likert_score_conversion.get(survey_label, False),
            }
        )

survey_df = pd.DataFrame(survey_data)
survey_data_path = ""
survey_df.to_json(os.path.join(OUTPUT_DIR, survey_data_path), lines=True, orient="records")

In [15]:
# Analyze survey data

conditions = survey_df.apply(lambda row: f"{row['experiment_id']}_{row['split_id']}", axis=1).unique()
subjective_results = ""
with open(subjective_results, "w") as file:
    file.write(f"total conditions: {len(conditions)}\n\n")

    for condition in conditions:
        experiment_id = condition.split('_')[0]
        split_id = condition.split('_')[1]
        condition_df = survey_df[(survey_df['experiment_id'] == experiment_id) & (survey_df['split_id'] == int(split_id))]
        file.write(f"===== {experiment_id}:{split_id} ======\n")

        survey_qns = [i for i in range(1, 17)]
        if split_id == '1': 
            survey_qns = [i for i in range(1, 17) if i not in range(2, 17)]
        if split_id == '2': 
            survey_qns = [i for i in range(1, 17) if i not in [4, 10, 11, 12, 13, 14]]

        for qn in survey_qns:
            tmp_df = condition_df[(condition_df['question_number'] == qn)]
            mean = np.mean(tmp_df['survey_label_score'])
            std = np.std(tmp_df['survey_label_score'])

            file.write(f"{qn}. mean: {round(mean, 3)}, std: {round(std, 3)}\n")
        file.write("\n")